# **Ollama와 llama_index를 활용한 RAG 기본 예제**

**Ollama**, **llama_index** 등의 라이브러리를 이용해 **Open-source LLM**으로 기본적인 **RAG** 기능을 구현하는 예제입니다.

- Ollama : LLM 모델 
- llama_index : RAG 문서 데이터셋
- sentence_transformers : 텍스트 임베딩 모델


*본 예제는 구글 코랩 환경이 아닙니다.

## **1. 환경 세팅**
- 개발 환경에 필요 라이브러리 설치 : ollama, langchain, langchain_community,...
- 필요 패키지 임포트
- Ollama 모델 사용 방법
- `search_related_docs`
  - 질의와 관련된 문서를 검색하는 함수
  - query 임베딩을 생성하고 유사도가 높은 문서를 찾음

In [29]:
from langchain.llms import Ollama
from llama_index.core import Document
from sentence_transformers import SentenceTransformer, util
from IPython.display import Markdown, display

In [30]:
def search_related_docs(embedding_model, query, documents):
# 질의 임베딩 생성
    query_embedding = embedding_model.encode(query)
    # 인덱스를 사용하여 관련 문서 검색
    doc_embeddings = embedding_model.encode([doc.text for doc in documents])
    cosine_scores = util.pytorch_cos_sim(query_embedding, doc_embeddings)

    # 상위 N개의 관련 문서 인덱스 찾기
    top_k = 2
    top_results = cosine_scores[0].topk(top_k)

    # 관련 문서 추출
    related_docs = [documents[idx].text for idx in top_results[1]]

    return related_docs

### Ollama 모델 사용 방법
- [Ollama 레포지토리](https://github.com/ollama/ollama?tab=readme-ov-file) 참고하여 OS 별 Ollama 다운로드
- `ollama pull {model_name}` 명령어로 로컬에 모델 다운로드
- 본 예제에서는 "gemma2:2b" 모델 사용 
- Ollama 레포지토리에서 모델 라이브러리를 참고하여 원하는 모델 다운로드 가능

## **2. 모델 로드**
- Ollama를 이용한 오픈소스 LLM 로드
- `SentenceTransformer` 의 텍스트 임베딩 모델 로드
  - 한국어 임베딩이 가능한 "distiluse-base-multilingual-cased-v1" 모델 사용

In [31]:
# Ollama 모델 로드
llm = Ollama(model='gemma2:2b')  # 사용할 Ollama 모델 선택

# 텍스트 임베딩 모델 로드
# distiluse-base-multilingual-cased-v1 : 한국어 임베딩을 지원하는 모델
# embedding_model = SentenceTransformer('all-mpnet-base-v2')
embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## **3. RAG 문서 데이터셋 정의**
- RAG에서 검색할 텍스트 데이터셋 정의
- `Documents` 객체를 생성하여 각 텍스트를 문서로 변환
- 예제에서는 윷놀이, 제기차기, 식혜 만들기, 스타크래프트에 대한 문서 사용

In [32]:
text_0 = """
윷놀이 규칙
1. 준비물

윷: 앞뒤가 다른 네 개의 막대.
윷판: 원형 모양의 판, 중앙으로 가는 지름길이 있습니다.
말: 각 팀이 사용할 말 4개.
2. 플레이어

최소 2명에서 4명까지 팀을 이루어 플레이합니다.
3. 윷 던지기 윷은 네 개의 막대로 구성되어 있으며, 던진 결과에 따라 다음과 같은 점수를 얻습니다:

도: 1칸 이동 (1개만 뒤집힘)
개: 2칸 이동 (2개 뒤집힘)
걸: 3칸 이동 (3개 뒤집힘)
윷: 4칸 이동 (4개 모두 뒤집힘)
모: 5칸 이동 (모두 안 뒤집힘)
4. 말 이동

윷이나 모가 나오면 한 번 더 던질 수 있습니다.
말을 이동시키며, 자신의 말 4개가 모두 완주하면 승리합니다.
5. 말 업기

같은 팀의 말이 한 칸에 모이면 말 업기가 가능합니다. 업힌 말은 함께 이동합니다.
6. 잡기

상대방의 말을 잡으면, 상대방 말은 시작점으로 돌아갑니다. 잡은 플레이어는 한 번 더 윷을 던집니다.
7. 승리 조건

자신의 말 4개를 모두 완주시키면 승리합니다.
특별 규칙
지름길: 중앙으로 가는 지름길을 통해 빠르게 이동할 수 있습니다.
말의 이동 선택: 윷을 던질 때, 여러 개의 말 중 어느 말이 이동할지 선택할 수 있습니다.
윷놀이는 간단하면서도 전략적인 게임으로, 운과 실력을 모두 요구하는 한국 전통 놀이입니다."""

text_1 ="""
제기차기는 한국의 전통 놀이로, 주로 어린이들이 즐기는 활동입니다. 제기차기의 기본 규칙은 다음과 같습니다:

제기 만들기: 제기는 주로 종이나 천으로 만들어진 둥글고 평평한 물체입니다. 흔히 종이로 만든 제기를 사용하며, 발 아래에 적당한 무게가 있는 경우가 많습니다.

차기: 플레이어는 제기를 발로 차서 공중으로 띄웁니다. 제기가 땅에 떨어지기 전에 다시 차서 계속 공중에 띄워야 합니다.

점수 시스템: 제기를 공중에 얼마나 많이 띄우는지에 따라 점수를 매깁니다. 특정 횟수 이상을 차면 점수를 부여할 수 있습니다.

대결: 여러 명이 함께 제기차기를 하며, 서로의 제기를 방해하거나 더 높은 점수를 내는 방식으로 대결할 수 있습니다.

규칙 변형: 지역이나 그룹에 따라 제기차기 방식이나 규칙이 다를 수 있으며, 다양한 변형 규칙이 존재합니다.

제기차기는 신체 운동과 협동심을 기르는 데 도움이 되는 놀이입니다!
"""

text_2 ="""
식혜는 한국의 전통적인 단 음료로, 주로 명절이나 특별한 날에 즐겨 먹습니다. 식혜 만드는 방법은 다음과 같습니다:

재료
찹쌀: 1컵
물: 6컵
엿기름: 1/2컵 (또는 엿기름 가루)
설탕: 1/2컵 (취향에 따라 조절)
소금: 약간
계피: (선택 사항)
생강: (선택 사항)
만들기
찹쌀 준비: 찹쌀을 깨끗이 씻어 3-4시간 정도 불립니다. 불린 찹쌀은 체에 받쳐 물기를 빼둡니다.

엿기름 추출: 엿기름을 물에 담가 1시간 정도 우려냅니다. 우린 물은 따로 준비해 두고 엿기름은 체에 걸러서 버립니다.

찹쌀 찌기: 불린 찹쌀을 찜통에 넣고 20-30분 정도 쪄서 익힙니다. 완전히 익힌 후에는 식혀 둡니다.

조리: 큰 냄비에 엿기름 물, 물 6컵을 넣고 끓입니다. 끓어오르면 불을 줄이고, 식힌 찹쌀을 넣습니다. 약한 불에서 30분 정도 끓입니다.

설탕과 소금 추가: 끓인 후 설탕과 소금을 넣고, 다시 약한 불에서 10분 정도 더 끓입니다. 이때 계피나 생강을 추가하면 향이 더해집니다.

식히기: 식혜가 완성되면 체에 걸러서 고운 식혜를 만듭니다. 체에 걸러낸 찹쌀은 다른 요리에 활용할 수 있습니다.

냉장 보관: 식혜는 냉장고에 보관하면 더욱 맛있게 즐길 수 있습니다. 차갑게 해서 마시면 좋습니다!"""

text_3 ="""
스타크래프트(StarCraft)는 블리자드 엔터테인먼트가 개발한 실시간 전략(RTS) 게임으로, 1998년에 처음 출시되었습니다. 이 게임은 주로 미래의 우주 전쟁을 배경으로 하며, 세 가지 종족인 테란(Terran), 저그(Zerg), 프로토스(Protoss) 간의 전투를 중심으로 진행됩니다. 주요 특징은 다음과 같습니다:

주요 특징
종족:

테란: 인간 기반의 종족으로, 기계적 유닛과 다양한 건물을 사용합니다. 전략적인 방어와 기동성이 강점입니다.
저그: 생물학적 유닛으로 구성된 종족으로, 빠른 생산 속도와 대규모 군대를 특징으로 합니다. 유연한 전술이 가능합니다.
프로토스: 고도로 발달된 외계 종족으로, 강력한 기술과 방어력을 가진 유닛을 보유하고 있습니다. 자원 관리가 중요합니다.
게임 모드:

싱글 플레이어: 캠페인 모드에서 스토리를 따라 미션을 수행합니다.
멀티 플레이어: 다른 플레이어와 대결하거나 팀을 이루어 경쟁하는 방식입니다. e스포츠 대회도 활발하게 열립니다.
전략: 자원 관리, 유닛 생산, 기술 개발, 전투 전략 등을 결합하여 승리를 목표로 합니다. 각 종족의 특성을 잘 활용하는 것이 중요합니다.

e스포츠: 스타크래프트는 e스포츠의 선두주자로, 다양한 대회와 리그가 열리고 있으며, 많은 프로게이머들이 활동하고 있습니다.

확장팩: 스타크래프트는 이후 스타크래프트: 브루드 워와 같은 확장팩이 출시되어 새로운 유닛과 기능을 추가했습니다.

스타크래프트는 그 깊이 있는 전략성과 독창적인 세계관 덕분에 많은 사랑을 받고 있으며, RTS 장르의 대표작으로 자리잡고 있습니다.
"""

In [33]:
# 문서 데이터셋 생성
documents = [
    Document(text=text_0),
    Document(text=text_1),
    Document(text=text_2),
    Document(text=text_3)
]

## **4. 질의 및 문서 검색**
- 사용자 질의와 가장 유사한 참고 문서를 검색하고,
- 검색된 관련 문서를 바탕으로 LLM 모델 응답을 생성

### 스타크래프트 관련 질의

In [34]:
query = "스타크래프트 종족별 특징에 대해 알려줘"
related_docs = search_related_docs(embedding_model, query, documents)
print("질문:", query)
print("관련 문서:", related_docs)

질문: 스타크래프트 종족별 특징에 대해 알려줘
관련 문서: ['\n스타크래프트(StarCraft)는 블리자드 엔터테인먼트가 개발한 실시간 전략(RTS) 게임으로, 1998년에 처음 출시되었습니다. 이 게임은 주로 미래의 우주 전쟁을 배경으로 하며, 세 가지 종족인 테란(Terran), 저그(Zerg), 프로토스(Protoss) 간의 전투를 중심으로 진행됩니다. 주요 특징은 다음과 같습니다:\n\n주요 특징\n종족:\n\n테란: 인간 기반의 종족으로, 기계적 유닛과 다양한 건물을 사용합니다. 전략적인 방어와 기동성이 강점입니다.\n저그: 생물학적 유닛으로 구성된 종족으로, 빠른 생산 속도와 대규모 군대를 특징으로 합니다. 유연한 전술이 가능합니다.\n프로토스: 고도로 발달된 외계 종족으로, 강력한 기술과 방어력을 가진 유닛을 보유하고 있습니다. 자원 관리가 중요합니다.\n게임 모드:\n\n싱글 플레이어: 캠페인 모드에서 스토리를 따라 미션을 수행합니다.\n멀티 플레이어: 다른 플레이어와 대결하거나 팀을 이루어 경쟁하는 방식입니다. e스포츠 대회도 활발하게 열립니다.\n전략: 자원 관리, 유닛 생산, 기술 개발, 전투 전략 등을 결합하여 승리를 목표로 합니다. 각 종족의 특성을 잘 활용하는 것이 중요합니다.\n\ne스포츠: 스타크래프트는 e스포츠의 선두주자로, 다양한 대회와 리그가 열리고 있으며, 많은 프로게이머들이 활동하고 있습니다.\n\n확장팩: 스타크래프트는 이후 스타크래프트: 브루드 워와 같은 확장팩이 출시되어 새로운 유닛과 기능을 추가했습니다.\n\n스타크래프트는 그 깊이 있는 전략성과 독창적인 세계관 덕분에 많은 사랑을 받고 있으며, RTS 장르의 대표작으로 자리잡고 있습니다.\n', '\n제기차기는 한국의 전통 놀이로, 주로 어린이들이 즐기는 활동입니다. 제기차기의 기본 규칙은 다음과 같습니다:\n\n제기 만들기: 제기는 주로 종이나 천으로 만들어진 둥글고 평평한 물체입니다. 흔히 종이로 만든 제기를 사용하며, 발 아래에 적당한 무게가 있는 경우가 많

In [35]:
response = llm.predict(f"질문: {query}\n관련 문서: {', '.join(related_docs)}\n응답:")
Markdown(response)

## 스타크래프트 종족별 특징 및 정보  

**1. 테란 (Terran):** 인간 종족으로, **기계적 유닛과 다양한 건물을 활용**합니다. 🤖💪 

   * **강점:** 전략적인 방어와 기동성이 우수하며, 여러 가지 기술과 유닛들을 사용하여 적극적으로 공격하고 방어할 수 있습니다.
   * **특징:**
      *  **인간적인 지휘**: 인공지능을 활용한 다양한 기술로 전략적인 플레이와 실제 상황을 고려하는 경험이 풍부합니다. 🧠 
      *  **기계적 강점**: 주력 유닛은 기계적으로 효율적인 공격과 지원 기능을 가집니다.
      * **대규모 군사:** 다양한 전문 지휘관을 통해 병사의 효율성과 공격력을 높여줍니다. ⚔️

**2. 저그 (Zerg):** 생물학적 유닛으로 구성된 종족입니다. 🦠💥  
   * **강점:** 빠른 생산 속도와 대규모 군대를 특징으로 합니다. ⚡️💪 
   * **특징:**
      * **생물학적 강점**: 공격적인 전투 스타일을 통해 적의 유닛을 파괴하고 진행할 수 있습니다.  💥 
      * **공격력**: 생리적으로 능률적이고 폭발적인 공격력으로 상대방을 빠르게 공격합니다. 💣 

**3. 프로토스 (Protoss):** 고도로 발달된 외계 종족입니다. 🌟🌌  
   * **강점:** 강력한 기술과 방어력을 가진 유닛을 보유하고 있습니다. 🛡️💪 
   * **특징:**
      * **지속적인 상대방**: 장기 전투를 통해 생존 확률을 높일 수 있습니다. ⏳
      * **자원 관리**:  전략적 조절과 끊임없는 개발 노력이 필요합니다. ⚙️ 

**4. 스타크래프트 종족 특징 요약:**

| 종족 | 유닛 & 기술 | 강점 | 특징 | 
|---|---|---|---| 
| 테란 | 기계적 유닛, 다양한 건물  | 전략적인 방어/기동성 | 인간 지휘와 기술을 활용해 승리 |
| 저그 | 생물학적 유닛 | 빠른 생산 속도 & 대규모 군대 | 공격적인 전투 스타일 | 
| 프로토스 | 강력한 기술, 방어력  | 지속적인 상대방 및 자원 관리 | 장기전투와 기술적인 우위 |


**5. 스타크래프트의 세계관:**

* **다양한 배경 & 역사**: 세 가지 종족 간의 전쟁과 이를 배경으로 한 이야기들이 있습니다.
* **자연스러운 대결**: 다른 종족들과의 격렬한 전투는 게임을 더욱 매력적인 경험으로 만들어줍니다.



**6. 스타크래프트는:**

* **RTS 장르의 선두주자:**  다양한 플레이어들을 위한 흥미로운 게임 경험을 제공합니다.
* **강력한 전략 게임:** 여러 가지 요소를 조절하고 전략적으로 진행해야 하는 게임입니다.


**7. 추가 정보**:

* 스타크래프트의 세계관은 매우 풍부하여 다양한 이야기를 통해  만성적인 경험을 제공합니다.
* 끊임없이 개발되는 스타크래프트는 장기적 관점에서도 게임의 지속 가능성이 높습니다.





### 윷놀이 관련 질의

In [36]:
query = "윷놀이에서 상대방 말을 잡으면 어떻게 돼?"
related_docs = search_related_docs(embedding_model, query, documents)
print("질문:", query)
print("관련 문서:", related_docs)

질문: 윷놀이에서 상대방 말을 잡으면 어떻게 돼?
관련 문서: ['\n윷놀이 규칙\n1. 준비물\n\n윷: 앞뒤가 다른 네 개의 막대.\n윷판: 원형 모양의 판, 중앙으로 가는 지름길이 있습니다.\n말: 각 팀이 사용할 말 4개.\n2. 플레이어\n\n최소 2명에서 4명까지 팀을 이루어 플레이합니다.\n3. 윷 던지기 윷은 네 개의 막대로 구성되어 있으며, 던진 결과에 따라 다음과 같은 점수를 얻습니다:\n\n도: 1칸 이동 (1개만 뒤집힘)\n개: 2칸 이동 (2개 뒤집힘)\n걸: 3칸 이동 (3개 뒤집힘)\n윷: 4칸 이동 (4개 모두 뒤집힘)\n모: 5칸 이동 (모두 안 뒤집힘)\n4. 말 이동\n\n윷이나 모가 나오면 한 번 더 던질 수 있습니다.\n말을 이동시키며, 자신의 말 4개가 모두 완주하면 승리합니다.\n5. 말 업기\n\n같은 팀의 말이 한 칸에 모이면 말 업기가 가능합니다. 업힌 말은 함께 이동합니다.\n6. 잡기\n\n상대방의 말을 잡으면, 상대방 말은 시작점으로 돌아갑니다. 잡은 플레이어는 한 번 더 윷을 던집니다.\n7. 승리 조건\n\n자신의 말 4개를 모두 완주시키면 승리합니다.\n특별 규칙\n지름길: 중앙으로 가는 지름길을 통해 빠르게 이동할 수 있습니다.\n말의 이동 선택: 윷을 던질 때, 여러 개의 말 중 어느 말이 이동할지 선택할 수 있습니다.\n윷놀이는 간단하면서도 전략적인 게임으로, 운과 실력을 모두 요구하는 한국 전통 놀이입니다.', '\n제기차기는 한국의 전통 놀이로, 주로 어린이들이 즐기는 활동입니다. 제기차기의 기본 규칙은 다음과 같습니다:\n\n제기 만들기: 제기는 주로 종이나 천으로 만들어진 둥글고 평평한 물체입니다. 흔히 종이로 만든 제기를 사용하며, 발 아래에 적당한 무게가 있는 경우가 많습니다.\n\n차기: 플레이어는 제기를 발로 차서 공중으로 띄웁니다. 제기가 땅에 떨어지기 전에 다시 차서 계속 공중에 띄워야 합니다.\n\n점수 시스템: 제기를 공중에 얼마나 많이 띄우는지에 따라 점수를 매깁니다. 특정 

In [37]:
response = llm.predict(f"질문: {query}\n관련 문서: {', '.join(related_docs)}\n응답:")
Markdown(response)

윷놀이는 상대방 말을 잡으면 **상대방의 말은 시작점으로 돌아갑니다.**  그리고 잡은 플레이어는 한 번 더 윷을 던집니다. 




**추가 정보:** 


* 잡기는 윷놀이에서 중요한 요소입니다.
* 잡기를 통해 상대방의 말을 획득하여 점수를 얻는 방법을 학습하고 실력을 향상할 수 있습니다. 



